# Term project : Milestone 4
API Data pull
### Yograj Karki

For this project, I'm going to pull historical weather data from Source: https://openweathermap.org/api

In [1]:
# Loading libraries
import pandas as pd
import requests
import json
import datetime as dt
import numpy as np

In [2]:
# API key
api_key= "ec160a45a9812a127cc354d8a5adea88"      

In [3]:
# Importing sampled accidents data file
df = pd.read_csv("sample_accidents.csv")
df.shape

(145000, 47)

In [4]:
df.head()

,ID,Severity,Start_Time,End_Time,Start_Lat,Start_Lng,End_Lat,End_Lng,Distance(mi),Description,...,Roundabout,Station,Stop,Traffic_Calming,Traffic_Signal,Turning_Loop,Sunrise_Sunset,Civil_Twilight,Nautical_Twilight,Astronomical_Twilight
0,A-30,2,2019-11-25 07:11:55,2019-11-25 08:42:16,43.220390,-85.500961,43.220390,-85.500961,0.0,Accident on 17 Mile Rd at Stout Ave.,...,False,False,False,False,False,False,Night,Night,Day,Day
1,A-44,2,2016-12-29 07:35:34,2016-12-29 08:05:17,28.698406,-82.451477,28.698406,-82.451477,0.0,Accident on County Hwy-480 Oak Park Blvd at Co...,...,False,False,False,False,False,False,Day,Day,Day,Day
2,A-67,2,2019-04-11 12:20:07,2019-04-11 12:49:16,31.759741,-106.484001,31.759741,-106.484001,0.0,Accident on Campbell St at Myrtle Ave.,...,False,False,False,False,True,False,Day,Day,Day,Day
3,A-107,2,2017-08-22 10:53:29,2017-08-22 11:23:16,41.256569,-95.940956,41.256569,-95.940956,0.0,Accident on 19th St Eastbound at Harney St.,...,False,True,False,False,True,False,Day,Day,Day,Day
4,A-123,2,2019-06-19 17:55:06,2019-06-19 18:24:54,33.713829,-112.285217,33.713829,-112.285217,0.0,Accident on 107th Ave at Happy Valley Pkwy.,...,False,False,False,False,True,False,Day,Day,Day,Day


In [5]:
# creating a new data frame for weather data
df2 = df[["ID","Start_Time","City"]].copy()
df2.head()

,ID,Start_Time,City
0,A-30,2019-11-25 07:11:55,Cedar Springs
1,A-44,2016-12-29 07:35:34,Homosassa
2,A-67,2019-04-11 12:20:07,El Paso
3,A-107,2017-08-22 10:53:29,Omaha
4,A-123,2019-06-19 17:55:06,Peoria


#### Openweathermap API requires date to be UNIX timestamp format.

In [6]:
# Creating a new column of epoch time from Start_Time
df2["Start_Time"] = pd.to_datetime(df["Start_Time"])

# converting datetime object to epoch unit
df2["epoch"] = df2['Start_Time'].astype('int64')//1e9
df2.head()

,ID,Start_Time,City,epoch
0,A-30,2019-11-25 07:11:55,Cedar Springs,1574665915
1,A-44,2016-12-29 07:35:34,Homosassa,1482996934
2,A-67,2019-04-11 12:20:07,El Paso,1554985207
3,A-107,2017-08-22 10:53:29,Omaha,1503399209
4,A-123,2019-06-19 17:55:06,Peoria,1560966906


In [7]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 145000 entries, 0 to 144999
Data columns (total 4 columns):
 #   Column      Non-Null Count   Dtype         
---  ------      --------------   -----         
 0   ID          145000 non-null  object        
 1   Start_Time  145000 non-null  datetime64[ns]
 2   City        144994 non-null  object        
 3   epoch       145000 non-null  int64         
dtypes: datetime64[ns](1), int64(1), object(2)
memory usage: 4.4+ MB


In [8]:
# Creating a function to retrieve weather data by City name and date
def get_weather_data(city, epoch):
        URL='http://history.openweathermap.org/data/2.5/history/city?q={},US&type=hour&start={}&appid=ec160a45a9812a127cc354d8a5adea88"'.format(city,epoch)
        data = requests.get(URL).json()
        
        humidity = data['main']['humidity']
        wind_speed = data['wind']['speed']
        wind_degree = data['wind']['deg']
        visibility = data['visibility']
        m1 = data['weather'][0]['description']
        m2 = data['weather'][0]['main']
        body = '{} {}'.format(m1, m2)

        return humidity, wind_speed, wind_degree, visibility, body


In [9]:
# Test call
city = "Boston"
country_code = "US"
start_time = 1560992106.0

url= "http://history.openweathermap.org/data/2.5/history/city?q={},{}&type=hour&start={}&appid={}".format(city,country_code, start_time, api_key)

data = requests.get(url).json()
data


{'cod': 401,
 'message': 'Invalid API key. Please see http://openweathermap.org/faq#error401 for more info.'}

### Apparently, API seems to be invalid for historical weather data but it works for current weather and forecasted weather data.

In [10]:
url = "http://api.openweathermap.org/data/2.5/weather?q=Richmond,CA,US&appid=ec160a45a9812a127cc354d8a5adea88"
data = requests.get(url).json()
data

{'coord': {'lon': -122.3477, 'lat': 37.9358},
 'weather': [{'id': 800,
   'main': 'Clear',
   'description': 'clear sky',
   'icon': '01n'}],
 'base': 'stations',
 'main': {'temp': 288.24,
  'feels_like': 287.95,
  'temp_min': 285.46,
  'temp_max': 290.45,
  'pressure': 1018,
  'humidity': 82},
 'visibility': 10000,
 'wind': {'speed': 3.58, 'deg': 253, 'gust': 4.92},
 'clouds': {'all': 9},
 'dt': 1627889466,
 'sys': {'type': 2,
  'id': 2017679,
  'country': 'US',
  'sunrise': 1627910019,
  'sunset': 1627960649},
 'timezone': -25200,
 'id': 5387428,
 'name': 'Richmond',
 'cod': 200}